In [1]:
import pandas as pd
Yi_elements = lambda Yi: 

def count_element_totals(Yi_state: pd.Series):
    element_totals = {}
    

SyntaxError: invalid syntax (721639231.py, line 2)

y    2
x    1
dtype: int64

In [2]:
Yi_col_names = ['YiCH4',
 'YiH',
 'YiO',
 'YiO2',
 'YiOH',
 'YiH2O',
 'YiHO2',
 'YiH2O2',
 'YiC',
 'YiCH',
 'YiCH2',
 'YiCH2(S)',
 'YiCH3',
 'YiH2',
 'YiCO',
 'YiCO2',
 'YiHCO',
 'YiCH2O',
 'YiCH2OH',
 'YiCH3O',
 'YiCH3OH',
 'YiC2H',
 'YiC2H2',
 'YiC2H3',
 'YiC2H4',
 'YiC2H5',
 'YiC2H6',
 'YiHCCO',
 'YiCH2CO',
 'YiHCCOH',
 'YiN',
 'YiNH',
 'YiNH2',
 'YiNH3',
 'YiNNH',
 'YiNO',
 'YiNO2',
 'YiN2O',
 'YiHNO',
 'YiCN',
 'YiHCN',
 'YiH2CN',
 'YiHCNN',
 'YiHCNO',
 'YiHOCN',
 'YiHNCO',
 'YiNCO',
 'YiC3H7',
 'YiC3H8',
 'YiCH2CHO',
 'YiCH3CHO',
 'YiN2',
 'YiAR']

['YiCH4',
 'YiH',
 'YiO',
 'YiO2',
 'YiOH',
 'YiH2O',
 'YiHO2',
 'YiH2O2',
 'YiC',
 'YiCH',
 'YiCH2',
 'YiCH2(S)',
 'YiCH3',
 'YiH2',
 'YiCO',
 'YiCO2',
 'YiHCO',
 'YiCH2O',
 'YiCH2OH',
 'YiCH3O',
 'YiCH3OH',
 'YiC2H',
 'YiC2H2',
 'YiC2H3',
 'YiC2H4',
 'YiC2H5',
 'YiC2H6',
 'YiHCCO',
 'YiCH2CO',
 'YiHCCOH',
 'YiN',
 'YiNH',
 'YiNH2',
 'YiNH3',
 'YiNNH',
 'YiNO',
 'YiNO2',
 'YiN2O',
 'YiHNO',
 'YiCN',
 'YiHCN',
 'YiH2CN',
 'YiHCNN',
 'YiHCNO',
 'YiHOCN',
 'YiHNCO',
 'YiNCO',
 'YiC3H7',
 'YiC3H8',
 'YiCH2CHO',
 'YiCH3CHO',
 'YiN2',
 'YiAR']

In [32]:
def decompose_name(name):
    name = name[2:]
    prev_letter = name[0]
    try: int(name[-1])
    except ValueError: name+='1' # normalize to flush last value
    element_count = {}
    for letter in name[1:]:
        try: count=int(letter); letter=None
        except ValueError: count=1
        
        if prev_letter:
            element_count[prev_letter]=element_count.setdefault(prev_letter, 0)+count
        prev_letter=letter
        print(element_count)
    return element_count

In [33]:
decompose_name('YiCH2CHO')

{'C': 1}
{'C': 1, 'H': 2}
{'C': 1, 'H': 2}
{'C': 2, 'H': 2}
{'C': 2, 'H': 3}
{'C': 2, 'H': 3, 'O': 1}


{'C': 2, 'H': 3, 'O': 1}

In [1]:
import yaml
import pandas as pd
import numpy as np

class ElementConservation:
    def __init__(self, mech_fn):
        

# verified to work: 11/2/22
def load_species_comp_from_mech(mech_fn):
    with open(mech_fn, 'r') as f:
        mech = yaml.load(f, yaml.BaseLoader) # BaseLoader needed so that "NO" is not loaded as a boolean...
    species_compositions = {'Yi'+species['name']: pd.Series(species['composition']).astype(int) for species in mech['species']}
    type_is_str = [type(k) is str for k in species_compositions.keys()]
    assert all(type_is_str)
    comp_df = pd.DataFrame(species_compositions).T
    comp_df = comp_df.fillna(0)
    return comp_df

def quantify_elements(Yi_state: pd.Series):
    # We can use this magic notation because when we transpose comp_df
    # Yi_state index & comp_df columns align which makes it easy to multiply
    # https://stackoverflow.com/questions/69226160/pandas-how-to-multiply-each-element-of-a-series-to-each-element-of-a-column-in
    return (comp_df.T*Yi_state.squeeze()).T.sum()

# Sanity Check/Unit Test (on 2 elements)
comp_df = load_species_comp_from_mech('../datasets/GRIMech30.yaml')
Yi_assert = pd.Series(np.zeros_like(comp_df.index), index=comp_df.index)
Yi_assert[['YiH', 'YiO2']] = np.abs(np.random.randn(2))
element_quantities = quantify_elements(Yi_assert)
assert (element_quantities[element_quantities!=0]==pd.Series({'H': Yi_assert.loc['YiH'], 'O': Yi_assert.loc['YiO2']*2})).all()

In [78]:
import pandas as pd
df = pd.read_csv('../datasets/2D_PMMA-Air_master.csv')

In [99]:
Yi_col_names = [col for col in df.columns if col.startswith('Yi')]
print(Yi_col_names)
assert all((col[2:] in species_comp.index for col in Yi_col_names))

Yi_data = df[Yi_col_names]

['YiH', 'YiO2', 'YiO', 'YiOH', 'YiH2', 'YiH2O', 'YiHO2', 'YiH2O2', 'YiCO', 'YiCO2', 'YiHCO', 'YiCH', 'YiT-CH2', 'YiCH3', 'YiCH2O', 'YiHCCO', 'YiC2H', 'YiCH2CO', 'YiC2H2', 'YiS-CH2', 'YiC2H4', 'YiCH3OH', 'YiCH2OH', 'YiCH3O', 'YiCH4', 'YiCH3O2', 'YiC2H3', 'YiC2H5', 'YiCH2CHO', 'YiCH3CHO', 'YiH2C2', 'YiC2H5O', 'YiN-C3H7', 'YiC2H6', 'YiC3H8', 'YiC3H6', 'YiC3H3', 'YiP-C3H4', 'YiA-C3H4', 'YiS-C3H5', 'YiC2H3CHO', 'YiA-C3H5', 'YiC2O', 'YiC4H4', 'YiCH3OCO', 'YiC3H2O', 'YiT-C3H5', 'YiC3H5O', 'YiC4H6', 'YiN-C4H5', 'YiI-C4H5', 'YiI-C3H7', 'YiHOCHO', 'YiCH3CHCO', 'YiCH3COCH2', 'YiC2H3CHCHO', 'YiCH3COCH3', 'YiCH3CO', 'YiI-C3H5CO', 'YiMP2D_C4H6O2', 'YiMP2J_C4H7O2', 'YiMP3J_C4H7O2', 'YiMMETHMJ_C5H7O2', 'YiMMETHAC_C5H8O2', 'YiMMETHPJ_C5H7O2', 'YiN2', 'YiAR']


In [102]:
#list(map(quantify_elements, Yi_data.iterrows()))

element_quantities = [quantify_elements(Yi_data.loc[i]) for i in Yi_data.index]

In [101]:
quantify_elements(Yi_data.loc[0])

Ar    0.000000
C     0.000000
H     0.000000
N     1.542785
O     0.457215
dtype: float64

In [92]:
s = Yi_data.loc[0]

In [103]:
element_quantities

[Ar    0.000000
 C     0.000000
 H     0.000000
 N     1.542785
 O     0.457215
 dtype: float64,
 Ar    1.110223e-16
 C     1.172765e-19
 H     8.134852e-20
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    1.110223e-16
 C     3.998081e-19
 H     2.773346e-19
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    0.000000e+00
 C     8.459587e-19
 H     5.868483e-19
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    1.110223e-16
 C     1.638535e-18
 H     1.136743e-18
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    1.110223e-16
 C     3.102714e-18
 H     2.152683e-18
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    1.110223e-16
 C     5.840115e-18
 H     4.052210e-18
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    0.000000e+00
 C     1.097747e-17
 H     7.617359e-18
 N     1.542785e+00
 O     4.572146e-01
 dtype: float64,
 Ar    1.110223e-16
 C     2.063364e-17
 H     1.431888e-17
 N     1.542785e+00
 O  